In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics import roc_curve, roc_auc_score, auc, matthews_corrcoef

import sys
import time
import pandas as pd
import math

from numpy import array, concatenate, dot, diag, mean, std, median


In [55]:
train_f = "DE_train.csv"

In [56]:


df_data = pd.read_csv(train_f)




In [57]:
Rec_m = list(df_data['RecA'])
Rec_m += list(df_data['RecB'])
Rec_v = []
for e in Rec_m:
    if e != "None":
        Rec_v.append(e)
        
Rec_v  = [float(x) for x in Rec_v]
Rec_m = median(Rec_v)



In [58]:
df_data.loc[df_data.RecA == 'None', 'RecA'] = Rec_m
df_data.loc[df_data.RecB == 'None', 'RecB'] = Rec_m


In [59]:
df_data

,ID,CADD1,CADD2,RecA,EssA,CADD3,CADD4,RecB,EssB,Path,DE,GP
0,dd071,1.816072,-1.000000,0.09548,1,3.761300,-1.000000,0.30578,1,1,UK,BBS2/MKKS
1,dd072,4.681720,7.089390,0.09548,1,3.708315,-1.000000,0.30578,1,1,UK,BBS2/MKKS
2,dd073,8.172637,-1.000000,0.09548,1,3.840462,-1.000000,0.30578,1,1,UK,BBS2/MKKS
3,dd074,8.172637,7.380028,0.09548,1,4.825142,-1.000000,0.30578,1,1,TD,BBS2/MKKS
4,dd075,4.073944,-1.000000,0.32774,1,4.169124,-1.000000,0.09548,1,1,TD,BBS4/BBS2
...,...,...,...,...,...,...,...,...,...,...,...,...
268,du90,3.380462,-1.000000,0.25081,0,2.170131,-1.000000,0.87681,0,1,DD,SMC1A/CFTR
269,du91,3.532409,-1.000000,0.58893,1,4.509158,-1.000000,0.58549,1,0,DD,AVPR2/SLC45A2
270,du93,4.317497,-1.000000,0.12564,1,5.559382,2.461791,0.13039,1,0,DD,HDAC8/IRX5
271,du94,4.229748,-1.000000,0.22678,0,7.031820,4.195388,0.6537,0,0,DD,NLGN4X/BCS1L


In [60]:
X = (array(df_data)[:,1:-2].astype(float))
y = array(
        df_data['DE'].replace(
            'TD', 2
        ).replace(
            'CO', 1
        ).replace(
            'UK', -1
        ).replace(
            'OV', 0
        ).replace(
            'DI', 0
        ).replace(
            'DD', 0
        )
    )

gene_pairs = array(df_data['GP'])
#    X_pred = array(df_prediction)[:,1:11]

In [63]:
gene_pairs

array(['BBS2/MKKS', 'BBS2/MKKS', 'BBS2/MKKS', 'BBS2/MKKS', 'BBS4/BBS2',
       'BBS1/BBS7', 'BBS1/BBS2', 'BBS1/MKKS', 'BBS1/MKKS', 'BBS1/BBS2',
       'BBS1/BBS4', 'BBS2/BBS4', 'BBS2/BBS4', 'BBS1/CCDC28B',
       'BBS1/CCDC28B', 'BBS7/BBS4', 'BBS7/BBS2', 'BBS10/BBS5',
       'BBS10/MKKS', 'BBS10/BBS12', 'BBS10/BBS12', 'BBS10/MKKS',
       'BBS12/BBS4', 'BBS1/BBS10', 'BBS1/BBS10', 'BBS2/MKKS', 'BBS2/MKKS',
       'BBS10/BBS2', 'BBS2/BBS12', 'BBS2/BBS12', 'BBS10/MKKS',
       'BBS10/BBS2', 'BBS10/MKKS', 'BBS1/BBS2', 'BBS1/TTC8', 'BBS1/BBS9',
       'BBS10/BBS12', 'BBS10/MKKS', 'BBS10/BBS2', 'ARL6/TTC8',
       'BBS7/TTC8', 'BBS1/BBS12', 'BBS1/CCDC28B', 'NEK1/DYNC2H1',
       'MYH7/RBM20', 'DSP/TTN', 'DSP/TTN', 'MYH7/NEXN', 'NEXN/TTN',
       'GDAP1/MFN2', 'GDAP1/MFN2', 'EMD/LMNA', 'GNRHR/FGFR1',
       'OTUD4/RNF216', 'KAL1/FGFR1', 'KAL1/PROKR2', 'GNRHR/PROK2',
       'SLC3A1/SLC7A9', 'SLC3A1/SLC7A9', 'SLC3A1/SLC7A9', 'SLC3A1/SLC7A9',
       'HAMP/HFE', 'HAMP/HFE', 'EDA/WNT10A', 'EDA/WNT

In [74]:

def getScores(pred, real, threshold=0.5):
        """ Returns sen/spe/acc/bac/pre/mcc """

        if len(pred) != len(real):
            raise Exception("ERROR: input vectors have differente len!")

        aucScore = roc_auc_score(real, pred)

        tp, fp, fn, tn = (
            sum(r == 0 and p  < threshold for r, p in zip(real, pred)),
            sum(r == 0 and p >= threshold for r, p in zip(real, pred)),
            sum(r == 1 and p  < threshold for r, p in zip(real, pred)),
            sum(r == 1 and p >= threshold for r, p in zip(real, pred))
        )

        sen = tp / (tp + fn)
        spe = tn / (tn + fp)
        mcc = (
            (tp * tn - fn * fp) / math.sqrt(
                (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
            )
        )

        return sen, spe, mcc, aucScore

def LOGO_crossValidation(X, y, groups, n_trees=100, n_epochs=50, thresholds=[1,1,1]):
        """
        Stratified cross-validation.
        X: Design matrix
        y: label vector
        groups: Gene pair vector to define training groups
        n_trees: Amount of trees in random forest
        n_epochs: number of cross validations to perform
        thresholds: see getScores
        """

        logo = LeaveOneGroupOut()
        clf = RandomForestClassifier(
            n_estimators=n_trees,
            max_depth=10,
            criterion='gini',
            min_samples_split=2,
            min_samples_leaf=2,
            bootstrap=True,
            n_jobs=1
        )

        # Vector to compute final scores
        sum_dd_se, sum_co_se, sum_td_se = [], [], []
        sum_dd_sp, sum_co_sp, sum_td_sp = [], [], []
        for i in range(n_epochs):

            start_time = time.time()
            values_t, values_p = [], []

            print("#"*10, "Trial %i" % i, "#"*10)
            # We leave one group out
            for train_index, test_index in logo.split(X, y, groups):

                X_fit, y_fit, X_train, y_train = (
                    X[train_index], y[train_index],
                    X[test_index],  y[test_index]
                )

                clf = clf.fit(X_fit, y_fit)

                y_predicted = clf.predict_proba(X_train)
                #print(y_predicted)

                # y_predicted is not shaped correctly. Reshape it to fit
                # getScores expectations.
                y_formatted = y_predicted[0]
                #for de in (0, 1, 2, 3):
                #    for i, proba in enumerate(y_predicted[de][:,1]):
                #        y_formatted[i][de] = proba
                # Predictions are concatenated into a prediction vector
                values_t, values_p = values_t + [yi for yi in y_train], values_p + [yi for yi in y_formatted]

            sen_dd, sen_co, sen_td, spe_dd, spe_co, spe_td = getScores(values_p, values_t, thresholds)

            sum_dd_se.append(sen_dd)
            sum_co_se.append(sen_co)
            sum_td_se.append(sen_td)
            sum_dd_sp.append(spe_dd)
            sum_co_sp.append(spe_co)
            sum_td_sp.append(spe_td)

            print('Duration:', round( (time.time() - start_time) * 100) / 100, 's')
            print('sen | dd - co - td / spe | dd - co - td')
            print('sen | ' + '-'.join(map(str, (sen_dd, sen_co, sen_td))) + ' / spe | ' + '-'.join(map(str, (spe_dd, spe_co, spe_td))))

In [75]:
LOGO_crossValidation(X,y, gene_pairs)

########## Trial 0 ##########


Exception: ERROR: input vectors have differente len!